# Targeted poisoning attack on MNIST dataset

## Import Libraries

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from matplotlib import pyplot as plt
import collections

import random
from tqdm import tqdm
import copy

# ignore info and warnings form tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Parameters declaration

In [4]:
n_clients = 50
n_dataset_epochs = 5
n_train_epochs = 50
batch_size = 20

client_learning_rate = 0.02
server_learning_rate = 1

mal_users_percentage = 0.2
# todo: could also be a list of values
target_value = 3
poisoned_value = 8

## Dataset Loading and manipulation

In [6]:
emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data()

### Preprocessing and organizing dataset

In [8]:
shuffle_buffer = 100
# todo change??
prefetch_buffer = 100

In [10]:
def batch_format(element):
    # flatten the images
    return collections.OrderedDict(
        x = tf.reshape(element['pixels'], [-1, 784]),
        y = tf.reshape(element['label'], [-1, 1]))

def preprocess(dataset):
    dataset = dataset.repeat(n_dataset_epochs)
    dataset = dataset.shuffle(shuffle_buffer, seed = 1)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(batch_format)
    # dataset = dataset.prefetch(prefetch_buffer)

    return dataset

In [12]:
def poison_dataset(dataset, target_honest, target_mal):
    # parse the dataset
    for batch in dataset:
        # get the labels of each batch and convert to numpy array
        labels = batch['y'].numpy()
        # itterate through each label
        for y in labels:
            # if we find the target label	
            if y == target_honest:
                y = target_mal
        batch['y'] = tf.convert_to_tensor(labels, dtype = tf.float32)
    # return the malicious dataset
    return dataset

In [14]:
## testing the preprocessing function

example_dataset = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0])
     
preprocessed_example_dataset = preprocess(example_dataset)

# for i in preprocessed_example_dataset:
#     labels = i['y'].numpy()
#     print(labels)
#     labels[0] = 1
#     print(labels)
#     tf_labels = tf.convert_to_tensor(labels, dtype = tf.float32)
#     print(tf_labels)
#     break

# print(preprocessed_example_dataset)

# sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
#                                      next(iter(preprocessed_example_dataset)))



In [16]:
def make_federated_data(client_data, client_ids):

    fed_data = []
    for id in client_ids:
        preprocessed_dataset = preprocess(client_data.create_tf_dataset_for_client(id))

        # prob = random.random()
        # mal% of the users are malicious
        # if prob < mal_users_percentage:  
        #     preprocessed_dataset = poison_dataset(preprocessed_dataset, target_honest, target_mal)              

        fed_data.append(preprocessed_dataset)

    return fed_data

## Model creation

In [18]:
def create_keras_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape = (784,)),
        tf.keras.layers.Dense(10, kernel_initializer = 'zeros'),
        tf.keras.layers.Softmax(),
    ])


In [20]:
def mnist_model():
    keras_model = create_keras_model()
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec = preprocessed_example_dataset.element_spec,
        loss = tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])

## Training

In [22]:
training_process = tff.learning.algorithms.build_weighted_fed_avg(
    mnist_model,
    client_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate = client_learning_rate),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate = server_learning_rate))

2023-06-07 13:52:20.776145: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'args_2' with dtype int32 and shape [?,1]
	 [[{{node args_2}}]]


In [24]:
train_state = training_process.initialize()


In [26]:
# build the process to have the model's architecture
evaluation_process = tff.learning.algorithms.build_fed_eval(mnist_model)

# initialize the state of the evaluation
evaluation_state = evaluation_process.initialize()
sample_test_clients = emnist_test.client_ids[n_clients : 2 * n_clients]
# test the model with the test data
# question: selection of clients during training??
federated_test_data = make_federated_data(emnist_test, sample_test_clients)

2023-06-07 13:52:24.841153: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'args_2' with dtype int32 and shape [?,1]
	 [[{{node args_2}}]]


In [ ]:
training_loss = []
training_acc = []
eval_loss = []
eval_acc = []

for epoch in range(1, n_train_epochs):

    print('Epoch: {:2d}'.format(epoch))
    #client selection, random, chosen from the first 100 clients
    clients = random.sample(emnist_train.client_ids[0:100], n_clients)
    # clients = emnist_train.client_ids[0:n_clients]
    
    #note: slow to converge with random clients, makes sense
    federated_train_data = make_federated_data(emnist_train, clients)

    # run a next on the training process to train the model
    result = training_process.next(train_state, federated_train_data)
    # update the model's state and get access to the metrics
    train_state = result.state
    train_metrics = result.metrics
    # print the training metrics
    training_acc.append(train_metrics['client_work']['train']['sparse_categorical_accuracy'])
    training_loss.append(train_metrics['client_work']['train']['loss'])

    print('Training accuracy: {:.3f}, Training loss: {:.3f}'.format(training_acc[-1], training_loss[-1]))

    # evaluate the model with test data

    # get weights from the trainged model
    model_weights = training_process.get_model_weights(train_state)
    # update the evaluation state with them
    evaluation_state = evaluation_process.set_model_weights(evaluation_state, model_weights)
    # run a next() to evaluate the model
    evaluation_output = evaluation_process.next(evaluation_state, federated_test_data)

    # get access to the evaluation metrics
    eval_metrics = evaluation_output.metrics['client_work']['eval']['total_rounds_metrics']

    eval_acc.append(eval_metrics['sparse_categorical_accuracy'])
    eval_loss.append(eval_metrics['loss'])
    # print the training metrics
    print('Testing accuracy: {:.3f}, Testing loss: {:.3f}\n\n'.format(eval_acc[-1], eval_loss[-1]))

In [15]:
print(evaluation_output.metrics)

NameError: name 'evaluation_output' is not defined

In [16]:
epochs = [x for x in range(1, n_train_epochs)]
plt.plot(epochs, training_acc)
plt.show()

ValueError: x and y must have same first dimension, but have shapes (49,) and (1,)